In [ ]:
# Advanced House Price Prediction using XGBoost

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Import XGBRegressor from XGBoost
from xgboost import XGBRegressor


In [ ]:
# Load the dataset
df = pd.read_csv("vietnam_housing_dataset.csv")
df.head()


Address  Area  Frontage  \
0  Dự án The Empire - Vinhomes Ocean Park 2, Xã L...  84.0       NaN   
1  Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...  60.0       NaN   
2  Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...  90.0       6.0   
3  Đường Nguyễn Văn Khối, Phường 11, Gò Vấp, Hồ C...  54.0       NaN   
4   Đường Quang Trung, Phường 8, Gò Vấp, Hồ Chí Minh  92.0       NaN   

   Access Road House direction Balcony direction  Floors  Bedrooms  Bathrooms  \
0          NaN             NaN               NaN     4.0       NaN        NaN   
1          NaN             NaN               NaN     5.0       NaN        NaN   
2         13.0      Đông - Bắc        Đông - Bắc     5.0       NaN        NaN   
3          3.5       Tây - Nam         Tây - Nam     2.0       2.0        3.0   
4          NaN      Đông - Nam        Đông - Nam     2.0       4.0        4.0   

       Legal status Furniture state  Price  
0  Have certificate             NaN   8.60  
1               NaN             NaN   7.50  
2     Sale contract             NaN   8.90  
3  Have certificate            Full   5.35  
4  Have certificate            Full   6.90

In [ ]:
# Drop rows with missing target (Price)
df = df.dropna(subset=["Price"])

# Define features and target
X = df.drop(columns=["Price"])
y = df["Price"]

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = X.select_dtypes(include=["object"]).columns

print("Numerical features:", list(numerical_cols))
print("Categorical features:", list(categorical_cols))


Numerical features: ['Area', 'Frontage', 'Access Road', 'Floors', 'Bedrooms', 'Bathrooms']
Categorical features: ['Address', 'House direction', 'Balcony direction', 'Legal status', 'Furniture state']


In [ ]:
# Preprocessing for numerical data: impute missing values with the median
numerical_transformer = SimpleImputer(strategy="median")

# Preprocessing for categorical data: impute with the most frequent value and then one-hot encode
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combine preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)


In [ ]:
# Define the XGBoost regressor
xgb_regressor = XGBRegressor(objective="reg:squarederror", random_state=42)

# Create the pipeline integrating the preprocessor and the XGBoost model
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", xgb_regressor)
])


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# Define a grid of hyperparameters to search
param_grid = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [3, 5, 7],
    "regressor__learning_rate": [0.01, 0.1, 0.2],
    "regressor__subsample": [0.8, 1.0]
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_absolute_error", verbose=1, n_jobs=-1)

# Fit on the training data
grid_search.fit(X_train, y_train)

# Output best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best MAE (neg):", grid_search.best_score_)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'regressor__learning_rate': 0.2, 'regressor__max_depth': 7, 'regressor__n_estimators': 200, 'regressor__subsample': 0.8}
Best MAE (neg): -1.3227875958765993


In [ ]:
# Use the best estimator found by grid search to predict on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate Mean Absolute Error on the test set
mae = mean_absolute_error(y_test, y_pred)
print(f"Test set Mean Absolute Error: {mae:.2f}")


Test set Mean Absolute Error: 1.34


In [ ]:
from joblib import dump

# Save the entire pipeline (preprocessor + model) to a file
dump(best_model, 'house_price_pipeline.joblib')

['house_price_pipeline.joblib']